In [94]:
from __future__ import division
from collections import defaultdict
from math import log

import pandas as pd
print(pd.__version__)
import numpy as np


# read datasets from .csv
df_women = pd.read_csv('case3women.csv', sep=';', error_bad_lines=False, encoding='cp1251')
df_women.columns = ['ID', 'X', 'NumberOfPersons', 'global_id', 'Year', 'Month', ' ']
label = ['ж']*len(df_women['X'])
df_women['Y'] = np.array(label)

df_men = pd.read_csv('case3men.csv', sep=';', error_bad_lines=False, encoding='cp1251')
df_men.columns = ['ID', 'X', 'NumberOfPersons', 'global_id', 'Year', 'Month', ' ']
label = ['м']*len(df_men['X'])
df_men['Y'] = np.array(label)

# create one common dataset for both men and women, clear and sort it
df = pd.concat([df_women, df_men])
df.drop(['ID', 'NumberOfPersons', 'global_id', 'Year', 'Month', ' '], axis=1, inplace=True)
for i in df['X']:
    if i.count(',') > 0 or i == 'Name':
        df = df.drop(df[df['X'] == i].index)

df = df[~df.X.duplicated(keep='first')]
df.sort_values('X', inplace=True, ascending=True)


def train(samples):
    classes, freq = defaultdict(lambda:0), defaultdict(lambda:0)
    for feats, label in samples:
        classes[label] += 1                 # count classes frequencies
        for feat in feats:
            freq[label, feat] += 1          # count features frequencies

    for label, feat in freq:                # normalize features frequencies
        freq[label, feat] /= classes[label]
    for c in classes:                       # normalize classes frequencies
        classes[c] /= len(samples)

    return classes, freq                    # return P(C) and P(O|C)

def classify(classifier, feats):
    classes, prob = classifier
    return min(classes.keys(),              # calculate argmin(-log(C|O))
        key = lambda cl: -log(classes[cl]) + \
            sum(-log(prob.get((cl,feat), 10**(-7))) for feat in feats))

def get_features(sample): return (
        'll: %s' % sample[-1],  # get last letter
        'pll: %s' % sample[-2], # get pre-last letter
        'fl: %s' % sample[0],  # get first letter           
        )

def test(classifier, test_set):
    hits = 0
    for feats, label in test_set:
        if label == classify(classifier, feats):
            hits += 1
    return hits/len(test_set)

features = [(get_features(x), y) for x, y in df.to_numpy()]
c = len(features)//2
train_set, test_set = features[:-c], features[-c:]

classifier = train(train_set)
print('Accuracy: ', test(classifier, test_set))

name = input()
print('gender: ', classify(classifier, get_features(name)))


0.23.4


AttributeError: 'DataFrame' object has no attribute 'to_numpy'